In [1]:
import pandas as pd
import numpy as np
import tqdm

import time
from datetime import datetime, timedelta
import pytz
tz = pytz.timezone('America/Mexico_City')



import json

import glob
import distutils.dir_util

# Run queries

In [2]:

lista = pd.read_csv("colima.csv")


In [3]:
len(lista)

185

In [4]:
l = lista.copy()

In [5]:
all_drivers = l.toolsdriver_partner_uuid.value_counts().to_frame().reset_index().rename(columns={'toolsdriver_partner_uuid': 'count_of_drivers', 'index': 'toolsdriver_partner_uuid'})

In [6]:
full = pd.merge(l, all_drivers, on='toolsdriver_partner_uuid')

In [7]:
l = full.copy()

# Make files
---

In [8]:
col_files = ['uuid', 'format', 'vehicle_owner', 'plate', 'driver', 'document']

#Driver docs
#ine_driver = l[['toolsdocument_identificacion_conductor', 'formato_identificacion_conductor', 'nombre_completo_socio', 'placa', 'nombre_completo_conductor']]
lic_driver = l[['toolsdocument_licencia_conducir_conductor', 'formato_licencia_conducir_conductor', 'nombre_propietario', 'placa', 'nombre_conductor']]
#cnap_driver = l[['toolsdocument_carta_no_antecedentes_conductor', 'formato_carta_no_antecedentes_conductor', 'nombre_completo_socio', 'placa', 'nombre_completo_conductor']]
#circ_files = l[['circ_uuid', 'circ_format']]


#Partner docs
#ine_partner = l[['toolsdocument_identificacion_socio', 'formato_identificacion_socio', 'nombre_completo_socio', 'placa', 'nombre_completo_conductor']]
#lic_partner = l[['toolsdocument_licencia_conducir_socio', 'formato_licencia_conducir_socio', 'nombre_completo_socio', 'placa', 'nombre_completo_conductor']]
#cnap_partner = l[['toolsdocument_carta_no_antecedentes_socio', 'formato_carta_no_antecedentes_socio', 'nombre_completo_socio', 'placa', 'nombre_completo_conductor']]


#Vehicle docs
circulacion = l[['toolsdocument_tarjeta_circulacion', 'formato_tarjeta_circulacion', 'nombre_propietario', 'placa', 'nombre_conductor']]
#seguro = l[['toolsdocument_seguro_vehicular', 'formato_seguro_vehicular', 'nombre_completo_socio', 'placa', 'nombre_completo_conductor']]


#Assign document type
#ine_driver = ine_driver.assign(document_type = 'ine_driver')
lic_driver = lic_driver.assign(document_type = 'lic_driver')
#cnap_driver = cnap_driver.assign(document_type = 'cnap_driver')
#ine_partner = ine_partner.assign(document_type = 'ine_partner')
#lic_partner = lic_partner.assign(document_type = 'lic_partner')
#cnap_partner = cnap_partner.assign(document_type = 'cnap_partner')
circulacion = circulacion.assign(document_type = 'circulacion')
#seguro = seguro.assign(document_type = 'seguro')


#Rename columns
#ine_driver.columns = col_files
lic_driver.columns = col_files
#cnap_driver.columns = col_files
#ine_partner.columns = col_files
#lic_partner.columns = col_files
#cnap_partner.columns = col_files
circulacion.columns = col_files
#seguro.columns = col_files
#circ_files.columns = col_files

d = pd.concat([lic_driver, circulacion])
#d = pd.concat([ine_files, lic_files, cnap_files, circ_files])

In [9]:
workname = 'Colima_{:%Y-%m-%d}'.format(datetime.now(tz))
print('Workfolder will be: {}'.format(workname))

Workfolder will be: Colima_2021-03-09


In [10]:
# create work folder
distutils.dir_util.mkpath('output_uber_s3/{}/'.format(workname))
distutils.dir_util.mkpath('input_uber_s3/{}/'.format(workname))
distutils.dir_util.mkpath('reports/{}/'.format(workname))
print('Created work {}'.format(workname))

Created work Colima_2021-03-09


## Chunk list of Docs

In [11]:
q = d.copy()
f = q.copy()

q = q.set_index('uuid')
q = q.sort_index()

f = f[['uuid', 'format']]
f = f[~f.uuid.duplicated(keep='first')]
f = f.dropna()

r = list(range(0,len(f), 30000))
r.append(len(f))

for i in range(len(r) -1):
    f[r[i]:r[i+1]].to_csv('input_uber_s3/{}/colima_csv_chunk_{}.csv'.format(workname, i), index=False)

Now, go to Alexandria and use the files generated under the `input_uber_s3` folder to generate the S3 links to downlaod the documents.


## Ingest S3 links

In [12]:
digest_list = glob.glob('output_uber_s3/{}/*.csv'.format(workname))

list_ = []
for i in digest_list:
    df = pd.read_csv(i)
    list_.append(df)
li = pd.concat(list_)
li = li[~li.UUID.duplicated(keep='first')]

len(q[(~q.index.isin(li.UUID))]) # number of files not found in the full list of documents

0

## Make Pickle

In [13]:
%%time
ll = li.to_dict(orient='records') 

for x,i in enumerate(ll):
    p = q.loc[i['UUID']]
    if isinstance(p, pd.core.series.Series):
        ll[x]['files'] = [p.to_dict()]
    else:
        ll[x]['files'] = p.to_dict(orient='records')

Wall time: 54 ms


In [14]:
pickle_ = 'reports/{}/colima_{:%Y-%m-%d}.pickle'.format(workname, datetime.now(tz))
pi = pd.to_pickle(ll, pickle_)

## Download!

Now, use the script `fetcher.py` to download the files to your computer (or hard drive). You will need to:
- Put the script in the same folder as the pickle file
- Open the Terminal and run the script. `python fetcher.py <pickle> <download folder>`
    - Example: `python fetcher.py the_pickle.p /Volumes/HDRIVE/Regulatory/GDL`
    - Hint: To get the path of folder, navigate to the folder and use the `pwd` to print the route

In [15]:
d = lista.copy()

In [16]:
eldic = d.to_dict(orient='records')
for e in eldic:
    e['dia'] = e['curp_conductor'][8:10]
    e['mes'] = e['curp_conductor'][6:8]
    if '00' in e['curp_conductor'][4:6] or '01' in e['curp_conductor'][4:6]:
        e['anio'] = '20' + e['curp_conductor'][4:6]
    else:
        e['anio'] = '19' + e['curp_conductor'][4:6]


In [17]:
for e in eldic:
    spl = e['nombre_conductor'].split(' ')
    if len(spl) == 3:
        e['nombre'] = spl[0]
        e['apellido_paterno'] = spl[1]
        e['apellido_materno'] = spl[2]
    elif len(spl) == 4:
        e['nombre'] = spl[0] + ' ' + spl[1]
        e['apellido_paterno'] = spl[2]
        e['apellido_materno'] = spl[3]
    else:
        e['nombre'] = 'Sacalo manual papus'

In [18]:
import requests
from bs4 import BeautifulSoup

In [19]:
url = 'https://www.consisa.com.mx/rfc'

In [20]:
dd = []

with tqdm.tqdm(total=len(eldic)) as bar:
    for e in eldic:
        if e['nombre'] == 'Sacalo manual papus':
            rfc = 'Sacalo manual mi ninio'
        else:
            try:
                payload = {'strNombre': e['nombre'],
                           'strPrimerApellido': e['apellido_paterno'],
                           'strSegundoApellido': e['apellido_materno'],
                           'strdia': e['dia'],
                           'strmes': e['mes'],
                           'stranio': e['anio'],
                           'task': 'calcular.rfc'
                          }

                req = requests.post(url, data=payload)
                soup = BeautifulSoup(req.text, 'html.parser')
                rfc = soup.find_all('strong')[4].find_next().string[1:14]
            
            except IndexError:
                rfc = 'Sacalo manual mi ninio'

        dec = {'toolsvehicle_vehicle_uuid': e['toolsvehicle_vehicle_uuid'],
               'placa': e['placa'], 
               'numero_de_serie': e['numero_de_serie'], 
               'marca': e['marca'],
               'submarca': e['submarca'],
               'modelo': e['modelo'],
               'nombre_propietario': e['nombre_propietario'],
               'nombre_conductor': e['nombre_conductor'],
               'numero_licencia_conductor': e['numero_licencia_conductor'],
               'rfc_conductor': rfc
              }
        dd.append(dec)
        bar.update(1)


100%|████████████████████████████████████████████████████████████████████████████████| 185/185 [02:16<00:00,  1.35it/s]


In [21]:
final = pd.DataFrame(dd)

In [22]:
final.replace('Sácalo manual mi ninio', 'Sacalo manual mi ninio', inplace=True)

In [23]:
final[['toolsvehicle_vehicle_uuid', 'placa', 'numero_de_serie', 'marca', 'submarca', 'modelo', 
       'nombre_propietario', 'nombre_conductor', 'numero_licencia_conductor', 
       'rfc_conductor'
      ]].to_excel('reports/{}/lista_de_conductores_uber_colima_{:%Y-%m-%d}.xlsx'.format(workname, datetime.now(tz)),encoding='UTF-8')


